We need to compile llama-cpp-python[server] using Nvidia Compile (nvcc). Make sure you are running a CUDA Notebook !

In [1]:
!NVCC_APPEND_FLAGS='-allow-unsupported-compiler' CUDACXX=/usr/local/cuda/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python[server] --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 135.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 168.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 175.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 162.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 212.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 194.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 161.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 187.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 228.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 224.7 MB/s eta 0

Next, install the hugging face CLI and Torch tool so we can grab a llama model to chat to.

In [2]:
pip install 'huggingface_hub[cli,torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 238.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Login to Hugging Face, you will need your API_KEY set as an environment variable. 

In [3]:
from huggingface_hub import login, logout
import os
login(os.getenv("HF_API_KEY"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


Download a llama-2 model. This one is quantized and described as "medium, balanced quality".

In [4]:
!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf to /opt/app-root/src/.cache/huggingface/hub/tmpssbcb5c9
llama-2-7b-chat.Q4_K_M.gguf: 100%|██████████| 4.08G/4.08G [00:10<00:00, 405MB/s]
./llama-2-7b-chat.Q4_K_M.gguf


Load the model making sure to specify to use the GPU with n_gpu_layers.

In [5]:
from llama_cpp import Llama
llm = Llama(model_path="llama-2-7b-chat.Q4_K_M.gguf", n_gpu_layers=-1)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

This is where we specify the completions.

In [6]:
output = llm("Q: Name the world's best 3 rugby teams? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)


llama_print_timings:        load time =      91.42 ms
llama_print_timings:      sample time =       4.50 ms /    32 runs   (    0.14 ms per token,  7106.37 tokens per second)
llama_print_timings: prompt eval time =      91.35 ms /    17 tokens (    5.37 ms per token,   186.09 tokens per second)
llama_print_timings:        eval time =     599.54 ms /    31 runs   (   19.34 ms per token,    51.71 tokens per second)
llama_print_timings:       total time =     760.49 ms /    48 tokens


And gather the output to print.

In [7]:
print(output['choices'])

[{'text': "Q: Name the world's best 3 rugby teams? A: 1. New Zealand All Blacks - They have won the most World Cups, including four recent titles in 2011, 201", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]


Now we can run llama_cpp.server and serve up the model.

In [ ]:
!python3 -m llama_cpp.server --model llama-2-7b-chat.Q4_K_M.gguf --n_gpu_layers=-1 --host=0.0.0.0 --port=8080

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

Try browsing to the Route and Proxy that your notebook exposes. Be sure to create the two Routes in OpenShift first.

```yaml
oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-openapi-json
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/openapi.json
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /openapi.json
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF

oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-v1
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/v1
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /v1
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF
```


Open in a browser tab and try it out using swagger.

https://jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com/notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/docs